In [ ]:
import pickle
import numpy as np
import scipy.io as sio
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.core import Activation, Dense, Flatten, Dropout
from keras.layers import Conv1D, AveragePooling1D, MaxPooling1D, BatchNormalization
from keras.utils import np_utils 
from sklearn.preprocessing import StandardScaler
from keras import backend as K
from keras.callbacks  import EarlyStopping
import matplotlib.pyplot as plt 
from sklearn.utils import class_weight
from keras import regularizers
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
################# define variables and set values for network parameters
#### network-related parameters
runNum = 3
nb_filters=[4, 4, 8]
hidden_neuron_num = 8
filter_size = 24
channels = 3
pool_size = 3
strides= 4
padding = 'same'
nb_epoch = [50,25] 
learn_rate = 1e-2 
batch_size = 120
mtm=0.9
nb_classes=5 ## number of classes is equal to the number of subjects
borderMode = 'same'
backend = 'tf'
n_splits = 5 ## 5-fold cross validation

##### evaluation metric (confusion matrix)
conf_matrix = np.zeros([nb_classes, nb_classes, n_splits, runNum])
conf_matrix_normalize = np.zeros([nb_classes, nb_classes, n_splits, runNum])

In [ ]:
#### path to the base directory
dataPath = 'path to data'
savePath = 'path to save results'

In [ ]:
###### load data and labels
matContent = sio.loadmat(dataPath + 'data.mat')
data = matContent['data']
labels = matContent['labels']

In [ ]:
kf = StratifiedKFold(n_splits = n_splits, shuffle = True) 
i = 0 ## fold counter

## train the model on 4 folds and test it on the remained fold 
for train, test in kf.split(data, labels):     
     trainingLabels = np_utils.to_categorical(labels[train], nb_classes) 
     print("TRAIN:", train)
     for r in range(runNum):
        # Prepare network inputs
        model = Sequential()
        model.add(Conv1D(filters=nb_filters[0], kernel_initializer='glorot_uniform', kernel_size=filter_size, padding=padding, activation='relu', input_shape =(data.shape[1], data.shape[2])))
        model.add(AveragePooling1D(pool_size=pool_size, strides=strides, padding=padding))
        model.add(BatchNormalization())        
        
        model.add(Conv1D(filters=nb_filters[1], kernel_size=filter_size, padding=padding, activation='relu', 
                 kernel_initializer='glorot_uniform')) 
        model.add(AveragePooling1D(pool_size=pool_size, strides=strides, padding=padding))
        model.add(BatchNormalization())
       
        model.add(Conv1D(filters=nb_filters[2], kernel_size=filter_size, padding=padding, activation='relu', 
                  kernel_initializer='glorot_uniform')) 
        model.add(AveragePooling1D(pool_size=pool_size, strides=strides, padding=padding))
        
        model.add(Flatten())
        model.add(Dense(hidden_neuron_num, kernel_initializer='glorot_uniform', activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(nb_classes))
        model.add(Activation('softmax'))
        
        earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
        for ep in range(len(nb_epoch)):
            optimizer = SGD(lr=learn_rate / 10 ** ep, decay=5e-4, momentum=0.9)
            model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
            model.fit(data[train], trainingLabels, batch_size=batch_size, 
                  epochs=nb_epoch[ep], verbose=2, callbacks=[earlyStopping], validation_split=0.1) 
                              
        ############## model prediction and evaluation ##############
       
        predicted_testLabels = model.predict_classes(data[test],verbose = 0)
        soft_targets_test = model.predict(data[test],verbose = 0)    
        conf_matrix_normalize[:,:,i, r] = confusion_matrix(labels[test], predicted_testLabels, normalize = 'true')
        conf_matrix[:,:,i, r] = confusion_matrix(labels[test], predicted_testLabels)
        sio.savemat(savePath  +'results.mat', {'conf_matrix':conf_matrix, 'conf_matrix_normalize':conf_matrix_normalize})
        
     i+=1 